In [1]:
import pandas as pd
from utils import Timer
import featuretools as ft
import pandas as pd
import numpy as np
from woodwork.logical_types import Categorical, Datetime, Double
from woodwork.column_schema import ColumnSchema
from woodwork.logical_types import Boolean, BooleanNullable
import lightgbm as lgbm

In [2]:
file = "/mnt/DP_disk1/ht/datasets/autofe/fraud_detect/card_transaction.v1.csv"
with Timer("read train data"):
    df =  pd.read_csv(file)
df['Amount'] = df['Amount'].str.replace('$', '')
df

read train data took 23.243142270483077 sec


/tmp/ipykernel_2912987/1420505378.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Amount'] = df['Amount'].str.replace('$', '')


,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
0,0,0,2002,9,1,06:21,134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750.0,5300,NaN,No
1,0,0,2002,9,1,06:42,38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
2,0,0,2002,9,2,06:22,120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,NaN,No
3,0,0,2002,9,2,17:45,128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754.0,5651,NaN,No
4,0,0,2002,9,3,06:23,104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750.0,5912,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,1,2020,2,27,22:23,-54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,NaN,No
24386896,1999,1,2020,2,27,22:24,54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054.0,5541,NaN,No
24386897,1999,1,2020,2,28,07:43,59.15,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,NaN,No
24386898,1999,1,2020,2,28,20:10,43.12,Chip Transaction,2500998799892805156,Merrimack,NH,3054.0,4121,NaN,No


In [3]:
feature_logical_types = {
    'User': Categorical, 
    'Card': Categorical,
    'Time': Datetime,
    'Amount': Double,
    'Use Chip': Categorical,
    'Merchant Name': Categorical,
    'Merchant City': Categorical,
    'Merchant State': Categorical,
    'Zip': Categorical,
    'MCC': Categorical,
    'Errors?': Categorical,
    'Is Fraud?': Boolean,
}

with Timer("Load data to entityset"):
    es = ft.EntitySet("fraud_detect")
    es.add_dataframe(dataframe_name="fraud_detect",
                     dataframe=df,
                     index="id", 
                     logical_types=feature_logical_types,
                     )

with Timer("DFS feature generation"):
    ret_df, features = ft.dfs(
        entityset=es, 
        target_dataframe_name="fraud_detect", 
        verbose=True,
        )

for col in ret_df.columns:
    if isinstance(ret_df[col].dtype, pd.BooleanDtype):
        ret_df[col] = ret_df[col].astype(bool)
    feature_name = col
    if '?' in feature_name:
        feature_name = feature_name.replace('?', '')
    if ' ' in feature_name:
        feature_name = feature_name.replace(' ', '_')
    ret_df = ret_df.rename(columns={col: feature_name})

ret_df.info()

/root/miniconda3/envs/autofe/lib/python3.8/site-packages/featuretools/entityset/entityset.py:1910: UserWarning: index id not found in dataframe, creating new integer column
  warnings.warn(
/root/miniconda3/envs/autofe/lib/python3.8/site-packages/featuretools/synthesis/deep_feature_synthesis.py:169: UserWarning: Only one dataframe in entityset, changing max_depth to 1 since deeper features cannot be created
  warnings.warn(


Load data to entityset took 49.92404803074896 sec
Built 18 features
Elapsed: 03:58 | Progress: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████
DFS feature generation took 238.83046394214034 sec
<class 'pandas.core.frame.DataFrame'>
Int64Index: 24386900 entries, 0 to 24386899
Data columns (total 18 columns):
 #   Column          Dtype   
---  ------          -----   
 0   User            category
 1   Card            category
 2   Year            int64   
 3   Month           int64   
 4   Day             int64   
 5   Amount          float64 
 6   Use_Chip        category
 7   Merchant_Name   category
 8   Merchant_City   category
 9   Merchant_State  category
 10  Zip             category
 11  MCC             category
 12  Errors          category
 13  Is_Fraud        bool    
 14  DAY(Time)       category
 15  MONTH(Time)     category
 16  WEEKDAY(Time)   category
 17  YEAR(Time)      category

In [4]:
params = {
    'boosting_type':'gbdt',
    'objective': 'binary',
    'num_leaves': 31,
    'learning_rate': 0.01,
    'seed':0,
    'verbose': 1,
    'metrics': 'auc'
}
target_label = 'Is_Fraud'
train_sample = ret_df[ret_df['Year'] < 2018]
valid_sample = ret_df[ret_df['Year'] == 2018]
lgbm_train = lgbm.Dataset(train_sample.drop(columns=[target_label]), train_sample[target_label])
lgbm_val = lgbm.Dataset(valid_sample.drop(columns=[target_label]), valid_sample[target_label])
with Timer("LGBM train"):
    model = lgbm.train(params=params, train_set=lgbm_train, valid_sets=lgbm_val, verbose_eval=100)

/root/miniconda3/envs/autofe/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Number of positive: 25179, number of negative: 20579668
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.335468 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 17894
[LightGBM] [Info] Number of data points in the train set: 20604847, number of used features: 17


/root/miniconda3/envs/autofe/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/root/miniconda3/envs/autofe/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.001222 -> initscore=-6.706049
[LightGBM] [Info] Start training from score -6.706049
[100]	valid_0's auc: 0.997604
LGBM train took 20.674518452957273 sec
